In [9]:
S_0 = float(input("Price of an asset today: "))
S_L = float(input("Lower price of an asset in future: "))
S_H = float(input("Higher price of an asset in future: "))

a = (S_H - S_0)/(S_H - S_L)
b = (-1) * a * S_L
print()
print(f"Replicating portofolio: a = {a} shares, b = {b}£ of cash")
C = a * (S_0 - S_L)
print(f"C = {C}£ - price of the option")

Price of an asset today: 100
Lower price of an asset in future: 80
Higher price of an asset in future: 120

Replicating portofolio: a = 0.5 shares, b = -40.0£ of cash
C = 10.0£ - price of the option


In [5]:
import math

def first_binomial_call(S_0, K, T, r, u, d, N):
    '''
    :param S_0: (£) current stock price
    :param K: (£) strike price
    :param T: (years) life of the option in years
    :param r: (float) risk-free interest rate in decimal
    :param u: (float > 1) increase in stock price
    :param d: (float < 1) decrease in stock price interest
    :param N: (int) number of time intervals in int
    :return C[(0,0)]: (£) price of European Call Option
    '''
    dt = T/N
    p = (math.exp(r * dt) - d)/(u - d)  # risk-neutral probability
    C = {}  # value of option at each node
    
    # calculating nodes at the last interval
    for k in range(N, -1, -1):  # k - number of stock's up movements
            S_k = S_0 * (u ** k) * (d ** (N-k))
            C[(N, k)] = max(S_k - K, 0)
            
    # moving backwards from last interval to first
    for n in range(N-1, -1, -1):  # n - depth in the tree
        for k in range(0, n + 1):
            df = math.exp(-r * dt)  # discount factor
            # expected value under risk-neutral measure
            EX = p * C[(n+1, k+1)] + (1-p) * C[(n+1, k)] 
            C[(n, k)] = df * EX
    return C[(0, 0)]

In [6]:
PRMS = {"S_0":100, "K":100, "T":1, "r":0, "u":1.2, "d":0.8, "N":1}
f"r = 0.00: price = {first_binomial_call(**PRMS):.2f}£"

'r = 0.00: price = 10.00£'

In [7]:
PRMS = {"S_0":100, "K":100, "T":1, "r":0.05, "u":1.2, "d":0.8, "N":1}
f"r = 0.05: price = {first_binomial_call(**PRMS):.2f}£"

'r = 0.05: price = 11.95£'

In [8]:
for N in [1,10,100,200,300,400,500]:
    v = first_binomial_call(S_0=100, K=100, T=1, r=0, u=1.2, d=0.8, N=N)
    print(f"N = {N:3d}: price = {v:.2f}")

N =   1: price = 10.00
N =  10: price = 25.62
N = 100: price = 68.55
N = 200: price = 84.56
N = 300: price = 91.90
N = 400: price = 95.61
N = 500: price = 97.58


In [10]:
def binomial_call(S_0, K, T, r, std, N):
    '''
    :param S_0: (£) current stock price
    :param K: (£) strike price
    :param T: (years) life of the option in years
    :param std: (float) standard deviation of the stock
    :param r: (float) risk-free interest rate in decimal
    :param N: (int) number of time intervals in int
    :return C[(0,0)]: (£) price of European Call Option
    '''
    dt = T/N
    u = math.exp(std * math.sqrt(dt))
    d = 1/u
    p = (math.exp(r * dt) - d)/(u - d)  # risk-neutral probability
    C = {}  # value of option at each node
    
    # calculating nodes at the last interval
    for k in range(N, -1, -1):  # k - number of stock's up movements
            S_k = S_0 * (u ** (2*k-N))
            C[(N, k)] = max(S_k - K, 0)
            
    # moving backwards from last interval to first
    for n in range(N-1, -1, -1):  # n - depth in the tree
        for k in range(0, n + 1):
            df = math.exp(-r * dt)  # discount factor
            # expected value under risk-neutral measure
            EX = p * C[(n+1, k+1)] + (1-p) * C[(n+1, k)] 
            C[(n, k)] = df * EX
    return C[(0, 0)]

In [12]:
for N in [1,10,100,200,300,400,500]:
    v = binomial_call(S_0=100, K=100, T=1, r=0, std=math.log(1.2)/math.sqrt(1), N=N)
    print(f"N = {N:3d}: price = {v:.2f}")

N =   1: price = 9.09
N =  10: price = 7.08
N = 100: price = 7.25
N = 200: price = 7.25
N = 300: price = 7.26
N = 400: price = 7.26
N = 500: price = 7.26
